In [1]:
#cm_sb_cdummies
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df
run_id = "d_2020_04_01"

/home/kyle/miniconda3/envs/views2/lib/python3.8/site-packages/libpysal/examples/remotes.py:26: UserWarning: Remote data sets not available. Check connection.
  warnings.warn("Remote data sets not available. Check connection.")
[2020-11-19 17:01:40,408] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:01:40,441] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:01:40,463] - views.utils.io:65 - DEBUG - Reading parquet at /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2020-11-19 17:01:40,951] - views.utils.io:72 - DEBUG - Finished reading parquet from /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [2]:
periods = get_periods(run_id) # as a list
periods_by_name = get_periods_by_name(run_id) # as a dict
period_a = periods_by_name["A"]
period_b = periods_by_name["B"]
period_c = periods_by_name["C"]
period_c

[2020-11-19 17:01:44,341] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:01:44,361] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml


Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)

In [3]:
model_from_pipeline_spec = all_cm_models_by_name["cm_sb_acled_protest"]
models=[model_from_pipeline_spec]

In [4]:
for model in models:
    model.periods = periods

In [9]:
model_from_pipeline_spec

{
  "name": "cm_sb_acled_protest",
  "col_outcome": "greq_25_ged_best_sb",
  "cols_features": [
    "splag_1_1_acled_count_pr",
    "time_since_acled_dummy_pr",
    "time_since_splag_1_1_acled_dummy_pr"
  ],
  "steps": [
    1,
    3,
    6,
    9,
    12,
    18,
    24,
    30,
    36,
    38
  ],
  "periods": [
    {
      "name": "A",
      "train_start": 121,
      "train_end": 396,
      "predict_start": 397,
      "predict_end": 432
    },
    {
      "name": "B",
      "train_start": 121,
      "train_end": 432,
      "predict_start": 433,
      "predict_end": 468
    },
    {
      "name": "C",
      "train_start": 121,
      "train_end": 480,
      "predict_start": 483,
      "predict_end": 520
    }
  ],
  "outcome_type": "prob",
  "estimators": {
    "name": "cm_sb_acled_protest",
    "initial_estimator": {
      "base_estimator": {
        "criterion": "gini",
        "splitter": "best",
        "max_depth": null,
        "min_samples_split": 2,
        "min_samples_leaf":

In [5]:
for model in models:
    model.fit_estimators(df, populate_extras = False)

[2020-11-19 17:04:35,965] - views.apps.model.api:441 - INFO - Fitting estimators for cm_sb_acled_protest
[2020-11-19 17:04:35,967] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_protest for period A step 1
[2020-11-19 17:04:35,982] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_protest
[2020-11-19 17:04:35,982] - views.apps.model.api:422 - DEBUG - cm_sb_acled_protest downsampled away 0
[2020-11-19 17:04:35,983] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_protest on 12399 rows
[2020-11-19 17:04:35,986] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_protest
[2020-11-19 17:04:52,870] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_protest A 1 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_1.joblib
[2020-11-19 17:05:03,818] - views.apps.model.api:119 - DEBUG - cm_sb_acled_protest saved to /home/kyle/code/Views2/Ope

[2020-11-19 17:08:31,940] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_protest
[2020-11-19 17:08:49,738] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_protest A 36 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_36.joblib
[2020-11-19 17:09:00,208] - views.apps.model.api:119 - DEBUG - cm_sb_acled_protest saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_36.joblib
[2020-11-19 17:09:00,219] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_protest for period A step 38
[2020-11-19 17:09:00,224] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_protest
[2020-11-19 17:09:00,225] - views.apps.model.api:422 - DEBUG - cm_sb_acled_protest downsampled away 0
[2020-11-19 17:09:00,225] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_protest on 10541 rows
[2020-11-19 17:09:00,225] - views.apps.model.ap

[2020-11-19 17:13:23,980] - views.apps.model.api:422 - DEBUG - cm_sb_acled_protest downsampled away 0
[2020-11-19 17:13:23,981] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_protest on 12777 rows
[2020-11-19 17:13:23,981] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_protest
[2020-11-19 17:13:43,822] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_protest B 30 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_30.joblib
[2020-11-19 17:13:56,543] - views.apps.model.api:119 - DEBUG - cm_sb_acled_protest saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_30.joblib
[2020-11-19 17:13:56,557] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_protest for period B step 36
[2020-11-19 17:13:56,566] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_protest
[2020-11-19 17:13:56,566] - views.apps.model.ap

[2020-11-19 17:18:48,397] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_protest for period C step 24
[2020-11-19 17:18:48,403] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_protest
[2020-11-19 17:18:48,404] - views.apps.model.api:422 - DEBUG - cm_sb_acled_protest downsampled away 0
[2020-11-19 17:18:48,404] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_protest on 15693 rows
[2020-11-19 17:18:48,404] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_protest
[2020-11-19 17:19:10,277] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_protest C 24 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_C_24.joblib
[2020-11-19 17:19:25,397] - views.apps.model.api:119 - DEBUG - cm_sb_acled_protest saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_C_24.joblib
[2020-11-19 17:19:25,409] - views.apps.model.ap

In [6]:
for model in models:
    df_pred = model.predict(df)
    # assign_into_df takes care to only overwrite rows with actual values
    # This way we can keep all periods in the same df
    # It's also idempotent, no joining, so run as many times as you like. 
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_a,
        period_test=period_b,)
        
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)
   

[2020-11-19 17:23:26,150] - views.apps.model.api:552 - INFO - Predicting for cm_sb_acled_protest
[2020-11-19 17:23:26,152] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_protest periods: [Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432), Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468), Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)]
[2020-11-19 17:23:26,214] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_1.joblib
[2020-11-19 17:23:32,136] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_3.joblib
[2020-11-19 17:23:37,858] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_A_6.joblib
[2020-11-19 17:23:43,619] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/co

[2020-11-19 17:27:29,633] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_1.joblib
[2020-11-19 17:27:35,814] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_3.joblib
[2020-11-19 17:27:42,298] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_6.joblib
[2020-11-19 17:27:48,824] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_9.joblib
[2020-11-19 17:27:55,217] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_12.joblib
[2020-11-19 17:28:01,496] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_protest_B_18.joblib
[2020-11-19 17:28:07,792] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views

[2020-11-19 17:30:40,524] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_protest step 30
[2020-11-19 17:30:40,535] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_protest step 36
[2020-11-19 17:30:40,547] - views.apps.model.api:611 - DEBUG - Calibrating cm_sb_acled_protest step 38
[2020-11-19 17:30:40,587] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_1_calibrated
[2020-11-19 17:30:40,594] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_3_calibrated
[2020-11-19 17:30:40,600] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_6_calibrated
[2020-11-19 17:30:40,606] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_9_calibrated
[2020-11-19 17:30:40,612] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_12_calibrated
[2020-11-19 17:30:40,619] - views.utils.data:110 - DEBUG - Inserting col ss_cm_sb_acled_protest_18_calibrated
[2020-11-19 17:30:40,625] - views.u

In [7]:
for model in models:
    model.evaluate(df)

[2020-11-19 17:31:30,517] - views.apps.model.api:966 - INFO - Evaluating cm_sb_acled_protest
[2020-11-19 17:31:30,518] - views.apps.model.api:970 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step-combined
[2020-11-19 17:31:30,555] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step 1
[2020-11-19 17:31:30,587] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step 3
[2020-11-19 17:31:30,617] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step 6
[2020-11-19 17:31:30,646] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step 9
[2020-11-19 17:31:30,675] - views.apps.model.api:1009 - DEBUG - Evaluating uncalibrated predictions for cm_sb_acled_protest period A step 12
[2020-11-19 17:31:30,706] - views.apps.model.api:1009 - DEBU

In [8]:
for model in models:
    print(model.name)
    #print(model.scores)
    print("EVAL SCORES:")
    print(json.dumps(model.scores, indent=2))
    print("FEATURE_IMPORTANCES")
    print(json.dumps(model.extras.feature_importances, indent=2))
    print("#"*80)

cm_sb_acled_protest
EVAL SCORES:
{
  "A": {
    "1": {
      "uncalibrated": {
        "average_precision": 0.12966455586189674,
        "area_under_roc": 0.6069412190808957,
        "brier": 0.09099977826200545
      },
      "calibrated": {}
    },
    "3": {
      "uncalibrated": {
        "average_precision": 0.12068828279154445,
        "area_under_roc": 0.5991967957048809,
        "brier": 0.09635594126693252
      },
      "calibrated": {}
    },
    "6": {
      "uncalibrated": {
        "average_precision": 0.0998410869189724,
        "area_under_roc": 0.5183513463335352,
        "brier": 0.10960297762406547
      },
      "calibrated": {}
    },
    "9": {
      "uncalibrated": {
        "average_precision": 0.11007233501761773,
        "area_under_roc": 0.5482348680145727,
        "brier": 0.09585489054560524
      },
      "calibrated": {}
    },
    "12": {
      "uncalibrated": {
        "average_precision": 0.10698565870471122,
        "area_under_roc": 0.540762846246792